![title](https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png)

# Specreduce Demo
**AAS 245 National Harbor, Jan 2025**<br>
**Author** : Camilla Pacifici, Space Telescope Science Institute

**Tutorial Overview**<br>
This tutorial will show the basic spectral extraction functionality in the Astropy package [Specreduce](https://specreduce.readthedocs.io/en/latest/). We will:

1. Download a 2D spectrum and look at it using Jdaviz/Specviz2d
2. Read the spectrum with Specutils/Spectrum1D
3. Calculate the trace 
4. Set up the background
5. Extract the 1D spectrum

## Import

In [ ]:
# Jdaviz to visualize the spectrum
from jdaviz import Specviz2d
# Matplotlib for other plotting
from matplotlib import pyplot as plt
# Astropy to load the spectrum
from astropy.io import fits
from astropy import units as u
from astropy.nddata import StdDevUncertainty
# Astropy modeling for fitting
from astropy.modeling.models import Polynomial1D, Spline1D
# Specutils to manipulate the spectrum
from specutils import Spectrum1D
# Specreduce methods for tracing, background, and extracting
from specreduce.tracing import FlatTrace, FitTrace
from specreduce.background import Background
from specreduce.extract import BoxcarExtract, HorneExtract

## Loading Data
Use the example spectrum provided or your own spectrum

In [ ]:
file = './hlsp_jades_jwst_nirspec_goods-n-mediumhst-00000804_clear-prism_v1.0_s2d.fits'

Load the spectrum in Specviz2d to take a look. As you can see, this is very convenient when you are working with a single spectrum.

In [ ]:
viz = Specviz2d()
viz.load_data(file)
viz.show()

## Load the spectrum using Specutils
If you are working with many spectra, you might want to use API methods. Here is a simple workflow.

In [ ]:
hdu = fits.open(file)
hdu.info()

In [ ]:
flux = hdu['FLUX'].data
fluxunit = u.Unit(hdu['FLUX'].header['GUNIT'])
fluxerr = hdu['FLUX_ERR'].data
fluxerrunit = u.Unit(hdu['FLUX_ERR'].header['UNIT'])
wave = hdu['WAVELENGTH'].data
waveunit = u.Unit(hdu['WAVELENGTH'].header['UNIT'])

spec2d = Spectrum1D(spectral_axis=wave*waveunit,
                    flux=flux*fluxunit,
                    uncertainty=StdDevUncertainty(fluxerr*fluxerrunit))

spec2d

## Find the trace
First simple flat, then fit...

In [ ]:
trace = FlatTrace(spec2d, 14)
trace.trace_pos

In [ ]:
plt.figure(figsize=(20,3))
plt.imshow(spec2d.flux, vmin=0, vmax=1E-20, origin='lower')
plt.hlines(trace.trace_pos, xmin=0, xmax=len(wave), color='red')
plt.show()

In [ ]:
# tracefit = FitTrace?
tracefit = FitTrace(spec2d, bins=50,
                    guess=trace.trace_pos,
                    window=10,
                    trace_model=Polynomial1D(1)
                   )
# tracefit.trace.data

In [ ]:
plt.figure(figsize=(20,3))
plt.imshow(spec2d.flux, vmin=0, vmax=1E-20, origin='lower')
plt.plot(tracefit.trace.data, color='red')
plt.show()

## Background
Because of the way JWST observes (nodding) the background is already subtracted. We show the calls, but we do not use them here
```
bg = Background.two_sided(spec2d, tracefit, separation=7, width=3)
subtracted_image = spec2d - bg
```

## Extraction
Bla bla...notes...

In [ ]:
# extract = BoxcarExtract?
extract = BoxcarExtract(spec2d,
                        tracefit,
                        width=5
                       )
spec1d = extract.spectrum
spec1d

In [ ]:
plt.plot(spec1d.spectral_axis, spec1d.flux)
plt.xlabel('wavelength')
plt.ylabel('flux')
plt.show()